<a href="https://colab.research.google.com/github/rurusasu/RecommendSystem/blob/main/RecBoleTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recboleを使って様々なレコメンドシステムをテストする

参考
* [RecBole を用いてクックパッドマートのデータに対する50以上のレコメンドモデルの実験をしてみた](https://techlife.cookpad.com/entry/2021/11/04/090000)
* [Atomic Files](https://recbole.io/docs/user_guide/data/atomic_files.html)
* [新しいデータセットの実行](https://recbole.io/docs/user_guide/usage/running_new_dataset.html#prepare-atomic-files)
* [RecBoleを使ってみよう3 Atomicファイルについて](https://zenn.dev/kentoo1/articles/d5aef1c67901a0)


In [1]:
# Googleドライブのマウント
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [7]:
#base_dir = "/content/drive/MyDrive/ColabNotebooks/RecBole"
base_dir = "/home/user/core/"

In [3]:
!pip install  --upgrade -q recbole ray kmeans_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 48.3 MB/s eta 0:00:00


## ライブラリ読み込み

In [4]:
import os
from typing import List

import pandas as pd
from recbole.quick_start import run_recbole

## データの読み込みとAtomic file の作成

### データの読み込み

In [8]:
# エクセルファイルからデータを読み込む
data = pd.read_excel(f"{base_dir}/data/sample_merged_full.xlsx")
data.head(2)

,user_id,target_id,rating,rating_conv,user_name_target,nickname_target,gender_target,location_target,age_range_target,height_range_target,...,body_type_user,personality_user,appearance_user,job_user,blood_type_user,car_user,interests_user,salary_user,plan_user,account_creation_timestamp_user
0,1,8627.0,0.0,1,原田遥,アオイ,女性,埼玉県伊奈町,45-49,150-154,...,スリム,元気,セクシー系,会社員,O型,有り,技術・プログラミング,8160000,option2,2024-01-14 00:11:34
1,1,18213.0,0.0,1,井上萌,ユイ,女性,福島県玉川村,30-34,150-154,...,スリム,元気,セクシー系,会社員,O型,有り,技術・プログラミング,8160000,option2,2024-01-14 00:11:34


### Atomic file 作成

In [9]:
class AtomicFileCreator:
    def __init__(self, data: pd.DataFrame, output_dir: str, atomic_file_name: str):
        """
        初期化関数。

        Args:
            data (pd.DataFrame): 入力データ。
            output_dir (str): 出力ディレクトリ。
            atomic_file_name (str): 原子ファイルの名前。
        """
        self.data = data
        self.output_dir = output_dir
        self.atomic_file_name = atomic_file_name
        os.makedirs(self.output_dir, exist_ok=True)

    def clean_data(self) -> pd.DataFrame:
        """
        データをクリーンアップし、不要なカラムを削除する。

        Returns:
            pd.DataFrame: クリーンアップされたデータ。
        """
        columns_to_drop = [
            'user_name_target',
            'nickname_target',
            'plan_target',
            'account_creation_timestamp_target',
            'user_name_user',
            'nickname_user',
            'plan_user',
            'account_creation_timestamp_user',
            #'date'
        ]
        cleaned_data = self.data.dropna().drop(columns=columns_to_drop)
        return cleaned_data

    def convert_data_types(self, data: pd.DataFrame) -> pd.DataFrame:
        """
        指定されたカラムのデータ型を変換する。

        Args:
            data (pd.DataFrame): 変換するデータ。

        Returns:
            pd.DataFrame: データ型が変換されたデータ。
        """
        float_columns = [
            'rating',
            'rating_conv',
            'salary_user',
            'salary_target',
            #'age_user',
            #'age_target'
        ]
        data[float_columns] = data[float_columns].astype(float)

        numeric_columns = data.select_dtypes(include=['number']).columns.tolist()
        int_columns = [col for col in numeric_columns if col not in float_columns]
        data[int_columns] = data[int_columns].astype(int)

        return data

    def create_user_file(self, data: pd.DataFrame) -> None:
        """
        ユーザーの原子ファイルを作成する。

        Args:
            data (pd.DataFrame): 原子ファイルを作成するためのデータ。
        """
        user_columns = [col for col in data.columns if '_user' in col] + ['user_id']
        user_df = data[user_columns].drop_duplicates(subset=['user_id'])
        user_column_types = self._get_column_types(user_df)
        user_df.columns = user_column_types
        user_df.to_csv(f'{self.output_dir}/{self.atomic_file_name}.user', index=False, sep='\t')

    def create_item_file(self, data: pd.DataFrame) -> None:
        """
        アイテムの原子ファイルを作成する。

        Args:
            data (pd.DataFrame): 原子ファイルを作成するためのデータ。
        """
        item_columns = [col for col in data.columns if '_target' in col] + ['target_id']
        item_df = data[item_columns].drop_duplicates(subset=['target_id'])
        item_column_types = self._get_column_types(item_df)
        item_df.columns = item_column_types
        item_df.to_csv(f'{self.output_dir}/{self.atomic_file_name}.item', index=False, sep='\t')

    def create_inter_file(self, data: pd.DataFrame) -> None:
        """
        インタラクションの原子ファイルを作成する。

        Args:
            data (pd.DataFrame): 原子ファイルを作成するためのデータ。
        """
        inter_df = data[['user_id', 'target_id', 'rating', 'rating_conv']]
        inter_df.columns = ['user_id:token', 'target_id:token', 'rating:float', 'rating_conv:float']
        inter_df.to_csv(f'{self.output_dir}/{self.atomic_file_name}.inter', index=False, sep='\t')

    def _get_column_types(self, df: pd.DataFrame) -> List[str]:
        """
        原子ファイル用のカラムタイプを生成する。

        Args:
            df (pd.DataFrame): カラムタイプを生成するデータフレーム。

        Returns:
            List[str]: カラムタイプのリスト。
        """
        float_columns = [
            'rating',
            'rating_conv',
            'salary_user',
            'salary_target',
            #'age_user',
            #'age_target'
        ]
        numeric_columns = df.select_dtypes(include=['number']).columns.tolist()
        int_columns = [col for col in numeric_columns if col not in float_columns]

        column_types = []
        for col in df.columns:
            if col in int_columns:
                column_types.append(f"{col}:token")
            elif col in float_columns:
                column_types.append(f"{col}:float")
            else:
                column_types.append(f"{col}:token_seq")
        return column_types

    def create_atomic_files(self) -> None:
        """
        全ての原子ファイルを作成するメイン関数。
        """
        cleaned_data = self.clean_data()
        converted_data = self.convert_data_types(cleaned_data)
        self.create_user_file(converted_data)
        self.create_item_file(converted_data)
        self.create_inter_file(converted_data)
        print("Atomic files have been created successfully.")


def load_pickle_data(file_path: str) -> pd.DataFrame:
    """
    データをpickleファイルから読み込む。

    Args:
        file_path (str): データファイルのパス。

    Returns:
        pd.DataFrame: 読み込んだデータ。
    """
    with open(file_path, 'rb') as file:
        data = pd.read_pickle(file)
    return data

def load_csv_data(file_path: str) -> pd.DataFrame:
    """
    データをCSVファイルから読み込む。

    Args:
        file_path (str): データファイルのパス。

    Returns:
        pd.DataFrame: 読み込んだデータ。
    """
    data = pd.read_csv(file_path)
    return data

def load_xlsx_data(file_path: str) -> pd.DataFrame:
    """
    データをExcelファイルから読み込む。

    Args:
        file_path (str): データファイルのパス。

    Returns:
        pd.DataFrame: 読み込んだデータ。
    """
    data = pd.read_excel(file_path)
    return data

In [11]:
atomic_file_name = 'profile'
input_file = f"{base_dir}/data/sample_merged_full.xlsx"
output_dir = f"{base_dir}/dataset/{atomic_file_name}"

data = load_xlsx_data(input_file)

atomic_file_creator = AtomicFileCreator(data, output_dir, atomic_file_name)
atomic_file_creator.create_atomic_files()

Atomic files have been created successfully.


# モデル設定
* [Model list](https://recbole.io/docs/user_guide/model_intro.html#context-aware-recommendation)

In [ ]:
model_list = [
    # General Recommendation
    'LDiffRec',
    'DiffRec',
    'Random',
    'NCL',
    'SimpleX',
    'NCEPLRec',
    'ADMMSLIM',
    'SGL',
    'SLIMElastic',
    'EASE',
    'RecVAE',
    'RaCT',
    'NNCF',
    'ENMF',
    'CDAE',
    'MacridVAE',
    'MultiDAE',
    'MultiVAE',
    'LINE',
    'DGCF',
    'LightGCN',
    'NGCF',
    'GCMC',
    'SpectralCF',
    'NAIS',
    'FISM',
    'DMF',
    'ConvNCF',
    # Context-aware Recommendation
    'EulerNet',
    'FiGNN',
    'KD_DAGFM',
    'AutoInt',
    'DCNV2',
    'DCN',
    'DIEN',
    'DIN',
    'WideDeep',
    'DSSM',
    'PNN',
    'FNN',
    'FwFM',
    'FFM',
    'AFM',
    'xDeepFM',
    'DeepFM',
    # Sequential Recommendation
    'FEARec',
    'CORE',
    'SINE',
    'LightSANs',
    'NPE',
    'HRM',
    'HGN',
    'RepeatNet',
    'SHAN',
    'FOSSIL',
    'KSR',
    'GRU4RecKG',
    'S3Rec',
    'FDSA',
    'SASRecF',
    'GRU4RecF',
    'GCSAN',
    'SRGNN',
    'BERT4Rec',
    'SASRec',
    'TransRec',
    'NextItNet',
    'Caser'
]

In [ ]:
if __name__ == "__main__":
    dataset = 'profile'

    parser = argparse.ArgumentParser()
    parser.add_argument("--model", "-m", type=str, default='LDiffRec', help="name of models")
    parser.add_argument(
        "--dataset", "-d", type=str, default=dataset, help="name of datasets"
    )
    parser.add_argument("--config_files", type=str, default=f"{base_dir}/config/profile.yml", help="config files")
    parser.add_argument(
        "--nproc", type=int, default=1, help="the number of process in this group"
    )
    parser.add_argument(
        "--ip", type=str, default="localhost", help="the ip of master node"
    )
    parser.add_argument(
        "--port", type=str, default="5678", help="the port of master node"
    )
    parser.add_argument(
        "--world_size", type=int, default=-1, help="total number of jobs"
    )
    parser.add_argument(
        "--group_offset",
        type=int,
        default=0,
        help="the global rank offset of this group",
    )

    args, _ = parser.parse_known_args()

    config_file_list = (
        args.config_files.strip().split(" ") if args.config_files else None
    )

    run(
        args.model,
        args.dataset,
        config_file_list=config_file_list,
        nproc=args.nproc,
        world_size=args.world_size,
        ip=args.ip,
        port=args.port,
        group_offset=args.group_offset,
    )

Evaluate   : 100%|█████████████████████████████████████████████████| 21/21 [00:00<00:00, 170.37it/s]


# クイックスタート

In [ ]:
config_dict = {
    'train_stage': 'actor_pretrain',
    'pretrain_epochs': 150,
    'train_neg_sample_args': None,
}
run_recbole(model='RaCT', dataset='convert_profile',
     config_dict=config_dict, saved=False)

In [ ]:
if __name__ == "__main__":
    model_list = [
        'RecVAE'
    ]

    dataset = 'convert_profile'

    for m_name in model_list:
        try:
            result = run_recbole(
                model=m_name,
                dataset=dataset,
                config_file_list = [f"{base_dir}/config/{dataset}.yml"],
            )

            #result = run_recbole(
            #    model=m_name,
            #    dataset=dataset,
                #config_file_list = [f"{base_dir}/config/ract_actor_{dataset}.yml"],
                #config_file_list = [f"{base_dir}/config/ract_critic_{dataset}.yml"],
                #config_file_list = [f"{base_dir}/config/ract_finetune_{dataset}.yml"],
            #)

            df_valid = pd.DataFrame([result["best_valid_result"]], index=['Validation'])
            df_test = pd.DataFrame([result["test_result"]], index=["Test"])

            # ディレクトリが存在しない場合は作成
            resutl_dir = f"{base_dir}/result/{dataset}/{m_name}"
            if not os.path.exists(resutl_dir):
                os.makedirs(resutl_dir)

            df_valid.to_csv(f"{resutl_dir}/valid.csv")
            df_test.to_csv(f"{resutl_dir}/test.csv")

        except Exception as e:
            print(f"Error occurred: {e}")
            continue

Evaluate   : 100%|███████████████████████████████████████████████| 996/996 [00:07<00:00, 129.88it/s]
